## VEP

In [1]:
import time
import glob

import glow
spark = glow.register(spark)
import pyspark.sql.functions as fx
from pyspark.sql.types import *
from pyspark.sql.functions import *
from random import sample
from pyspark.sql.functions import expr

In [2]:

df = spark.read\
    .format("vcf")\
    .load("data/CPCT02450007T_vs_CPCT02450007R.strelka.somatic.snvs.norm.PASS.vep.vcf.gz")

df.printSchema()

WARNING	2021-08-10 23:46:27	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
21/08/10 23:46:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- contigName: string (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- qual: double (nullable = true)
 |-- filters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- splitFromMultiAllelic: boolean (nullable = true)
 |-- INFO_HMF_OVA_AC: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_SomaticEVS: double (nullable = true)
 |-- INFO_QSS_NT: integer (nullable = true)
 |-- INFO_HMF_OVA: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_PNOISE2: double (nullable = true)
 |-- INFO_MQ0: integer (nullable = true)
 |-- INFO_ReadPosRankSum: double (nullable = true)
 |-- INFO_TQSS: integer (nullable = true)
 |-- INFO_SNVSB: double (null

In [3]:
df.count()

WARNING	2021-08-10 23:46:37	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


19582

In [4]:
df.select(size("INFO_CSQ")).groupby().sum().collect()

WARNING	2021-08-10 23:46:38	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


[Row(sum(size(INFO_CSQ))=67655)]

In [5]:
annodf = df.withColumn("CSQ", expr("explode(INFO_CSQ)")).drop('INFO_CSQ')
annodf.printSchema()

root
 |-- contigName: string (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- qual: double (nullable = true)
 |-- filters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- splitFromMultiAllelic: boolean (nullable = true)
 |-- INFO_HMF_OVA_AC: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_SomaticEVS: double (nullable = true)
 |-- INFO_QSS_NT: integer (nullable = true)
 |-- INFO_HMF_OVA: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_PNOISE2: double (nullable = true)
 |-- INFO_MQ0: integer (nullable = true)
 |-- INFO_ReadPosRankSum: double (nullable = true)
 |-- INFO_TQSS: integer (nullable = true)
 |-- INFO_SNVSB: double (null

In [6]:
annodf.count()

WARNING	2021-08-10 23:46:45	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


67655

In [7]:
# strip normal genotypes
annodf2 = annodf.withColumn("GENO", expr("explode(genotypes)")).drop('genotypes')
annodf2 = annodf2.filter(annodf2.GENO.sampleID == 'TUMOR')
annodf2.count()

WARNING	2021-08-10 23:46:47	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


67655

In [8]:
annodf2.printSchema()

root
 |-- contigName: string (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- qual: double (nullable = true)
 |-- filters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- splitFromMultiAllelic: boolean (nullable = true)
 |-- INFO_HMF_OVA_AC: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_SomaticEVS: double (nullable = true)
 |-- INFO_QSS_NT: integer (nullable = true)
 |-- INFO_HMF_OVA: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_PNOISE2: double (nullable = true)
 |-- INFO_MQ0: integer (nullable = true)
 |-- INFO_ReadPosRankSum: double (nullable = true)
 |-- INFO_TQSS: integer (nullable = true)
 |-- INFO_SNVSB: double (null

### Filter by impact

In [11]:
%%time
annodf2.select('contigName','start','CSQ', col('CSQ.SYMBOL').alias('SYMBOL'))\
       .filter('CSQ.IMPACT == "HIGH"').show()

WARNING	2021-08-10 23:47:35	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+----------+---------+--------------------+------------+
|contigName|    start|                 CSQ|      SYMBOL|
+----------+---------+--------------------+------------+
|         1|212003261|{G, [splice_donor...|RP11-552D8.1|
|         8|145770631|{A, [splice_donor...|    ARHGAP39|
|         8|145770631|{A, [splice_donor...|    ARHGAP39|
|         8|145770631|{A, [splice_donor...|    ARHGAP39|
|         8|145770631|{A, [splice_donor...|    ARHGAP39|
|        15| 77426053|{A, [stop_gained]...|       PEAK1|
|        15| 77426053|{A, [stop_gained]...|       PEAK1|
|        17|  6014150|{G, [stop_gained]...|       WSCD1|
|        17|  6014150|{G, [stop_gained]...|       WSCD1|
|        17|  6014150|{G, [stop_gained]...|       WSCD1|
|        17|  6014150|{G, [stop_gained]...|       WSCD1|
|        17|  6014150|{G, [stop_gained]...|       WSCD1|
|        17| 29546068|{A, [stop_gained]...|         NF1|
|        17| 29546068|{A, [stop_gained]...|         NF1|
|        17| 29546068|{A, [stop

In [10]:
## Nice way but generates Alias exception

a_df = df.withColumn("Exploded_INFO_CSQ", expr("explode(INFO_CSQ)")) \
        .selectExpr("contigName", "start", "end", \
                "referenceAllele", "alternateAlleles", \
                "expand_struct(Exploded_INFO_CSQ)", \
                "genotypes"\
               )

Py4JJavaError: An error occurred while calling o79.selectExpr.
: java.lang.NoSuchMethodError: 'void org.apache.spark.sql.catalyst.expressions.Alias.<init>(org.apache.spark.sql.catalyst.expressions.Expression, java.lang.String, org.apache.spark.sql.catalyst.expressions.ExprId, scala.collection.Seq, scala.Option)'
	at io.projectglow.sql.expressions.ExpandStruct.$anonfun$expand$1(glueExpressions.scala:45)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at io.projectglow.sql.expressions.ExpandStruct.expand(glueExpressions.scala:43)
	at io.projectglow.sql.optimizer.ResolveExpandStructRule$.$anonfun$expandExprs$1(hlsOptimizerRules.scala:82)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.